<a href="https://colab.research.google.com/github/FVargasData/wpgPermits/blob/Initial-Setup/Datathon2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import necessary libraries

In [ ]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import time
import logging

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Set seaborn pastel palette for visualizations

In [ ]:
sns.set_palette("pastel")
sns.set_style("whitegrid")

# Set logging configuration

In [ ]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# CONFIGURATION and API parameters

In [ ]:
API_URL = "https://data.winnipeg.ca/resource/urbd-qygv.json"
APP_TOKEN = "CvU5WTCBwn36S6Vxebw1QjJVd"
CHUNK_SIZE = 1000  # number of rows per request (Socrata default)
MAX_ROWS = 330000  # maximum rows expected

# Columns to be retained based on our analysis

In [ ]:
COLUMNS_TO_KEEP = [
    "issue_date", "permit_group", "permit_type", "sub_type", "work_type",
    "neighbourhood_name", "community", "ward", "application_received_date", "status",
    "x_coordinate_nad83", "y_coordinate_nad83"
]